# About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:
           1: Fake news
           0: real News





In [1]:
import numpy as np
from nltk.corpus import stopwords


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rudra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Data Preprocessing

In [4]:
import pandas as pd

In [5]:
#Loading news data from with the help of pandas
news_data= pd.read_csv("train.csv")


In [6]:
news_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
news_data.shape

(20800, 5)

In [8]:
# finding missing values in eacn column of data news_data
news_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

# Imputation of dataset

In [9]:
# imputing the null values with empty string
news_data = news_data.fillna('')

In [10]:
#merging author and title for efficient working of model
news_data['content'] = news_data['author']+' '+news_data['title']

In [11]:
print(news_data['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [12]:
# Separating label from data
X = news_data.drop(columns='label', axis=1)
Y = news_data['label']

In [13]:
Y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [14]:
X.head()

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...


# stemming:  Aprocess of finding root word from a given word in sample data
ex. creator>>create      dancing>>dance

In [15]:
from nltk.stem.porter import PorterStemmer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [16]:
port_stem = PorterStemmer()

In [17]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [18]:
news_data['content'] = news_data['content'].apply(stemming)

In [19]:
print(news_data['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [20]:
#separating the data and label
X = news_data['content'].values
Y = news_data['label'].values

In [21]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [22]:
print(Y)

[1 0 1 ... 0 1 1]


In [23]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [24]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

# Spliting news_data into train and test data

In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

# Training 1: Training with LogisticRegression model

In [26]:
model = LogisticRegression()

In [27]:
model.fit(X_train, Y_train)

LogisticRegression()

# Evalution with accuracy score

In [34]:
# accuracy score on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)
print('Accuracy score of the training data : ', training_data_accuracy)
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the training data :  0.9865985576923076
Accuracy score of the test data :  0.9790865384615385


# hyperparameter tuning

# With 30:70 ratio


In [35]:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, stratify=Y, random_state=2)

# accuracy score on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)
print('Accuracy score of the training data : ', training_data_accuracy)
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the training data :  0.986195054945055
Accuracy score of the test data :  0.9825320512820512


# With 40:60 ratio


In [39]:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.4, stratify=Y, random_state=2)

# accuracy score on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)
print('Accuracy score of the training data : ', training_data_accuracy)
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the training data :  0.9864583333333333
Accuracy score of the test data :  0.9830528846153846


# With 50:50 ratio


In [40]:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.5, stratify=Y, random_state=2)

# accuracy score on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)
print('Accuracy score of the training data : ', training_data_accuracy)
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the training data :  0.98625
Accuracy score of the test data :  0.9839423076923077


# With 60:40 ratio


In [41]:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.6, stratify=Y, random_state=2)

# accuracy score on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)
print('Accuracy score of the training data : ', training_data_accuracy)
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the training data :  0.9868990384615385
Accuracy score of the test data :  0.9838942307692308


# with 70:30 ratio


In [42]:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.7, stratify=Y, random_state=2)

# accuracy score on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)
print('Accuracy score of the training data : ', training_data_accuracy)
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the training data :  0.9860576923076924
Accuracy score of the test data :  0.984684065934066


# with 80:20 ratio

In [43]:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, stratify=Y, random_state=2)

# accuracy score on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)
print('Accuracy score of the training data : ', training_data_accuracy)
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the training data :  0.986195054945055
Accuracy score of the test data :  0.9825320512820512


# Making a predictive system

In [32]:
n=20800
for i in range(n):
    X_pre = X_test[i]
    prediction = model.predict(X_pre)
    print(prediction)
    if (prediction[0]==0):
        print('The news is Real')
    else:
        print('The news is Fake')


[1]
The news is Fake
[0]
The news is Real
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fake
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[1]
The news is Fake
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fake
[0]
The news is Real
[1]
The news is Fake
[0]
The news is Real
[1]
The news is Fake
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fake
[1]
The news 

IndexError: row index (4160) out of range

# Training 2: Training the data with KNN 


In [45]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,Y_test=train_test_split(
        X,Y,test_size=0.3,random_state=0)

print(np.shape(X))
print(np.shape(X_train))


from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=3,p=2)
knn.fit(X_train,Y_train)
Y_pred=knn.predict(X_test)
print('misclassified samples: %d'%(Y_test!=Y_pred).sum())#compute
from sklearn.metrics import accuracy_score
print('Accuracy:%.2f'%accuracy_score(Y_test,Y_pred))
n=20800
for i in range(n):
    X_pre = X_test[i]
    prediction = model.predict(X_pre)
    print(prediction)
    if (prediction[0]==0):
        print('The news is Real')
    else:
        print('The news is Fake')


(20800, 17128)
(14560, 17128)
misclassified samples: 3131
Accuracy:0.50
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fake
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[1]


IndexError: row index (6240) out of range

# Training 3: training the data with SVM

In [50]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=0)
print(np.shape(X))
print(np.shape(X_train))
print(np.shape(X_test))


from sklearn.svm import SVC
svm = SVC(kernel='linear', C=1, random_state=0)
svm.fit(X_train, Y_train)
Y_pred=svm.predict(X_test)

from sklearn.metrics import accuracy_score
print('Accuracy:%.2f'%accuracy_score(Y_test,Y_pred))
n=20800
for i in range(n):
    X_pre = X_test[i]
    prediction = model.predict(X_pre)
    print(prediction)
    if (prediction[0]==0):
        print('The news is Real')
    else:
        print('The news is Fake')

(20800, 17128)
(14560, 17128)
(6240, 17128)
Accuracy:0.99
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fake
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fa

IndexError: row index (6240) out of range

# Applying ensemble approach on the prepared model

In [49]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(
        X,Y,test_size=0.3,random_state=0)
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(criterion='entropy',
                                 n_estimators=10, 
                                 random_state=1,
                                 n_jobs=2)
forest.fit(X_train, Y_train)
Y_pred=forest.predict(X_test)
print('misclassified samples: %d'%(Y_test!=Y_pred).sum())#compute
from sklearn.metrics import accuracy_score
print('Accuracy:%.2f'%accuracy_score(Y_test,Y_pred))
n=20800
for i in range(n):
    X_pre = X_test[i]
    prediction = model.predict(X_pre)
    print(prediction)
    if (prediction[0]==0):
        print('The news is Real')
    else:
        print('The news is Fake')

misclassified samples: 77
Accuracy:0.99
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fake
[0]
The news is Real
[1]
The news is Fake
[1]
The news is Fake
[0]
The news is Real
[1]
The news is Fake
[0]
The news is Real
[0]
The news is Real
[1]
The news is Fake
[0]
The news is

IndexError: row index (6240) out of range